In [88]:
from dataclasses import dataclass, field
from typing import List, Literal
from datetime import datetime, timedelta

from database import SchedulerStorage
from model import Slot, Event, Scheduler

import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination



In [ ]:
def optimize_schedule( x:np.ndarray, slots:List[Slot], events:List[Event], debug:bool) -> tuple[float, int, float, List[Event], List[Slot]]:

    if debug: print(f'x: {x}');
    # extract x
    duration_scalars = x[:len(events)];
    ordering_values  = x[len(events):];
    
    # sort events by second half of x
    events_zip = list(zip(events, duration_scalars, ordering_values));
    events_zip.sort(key=lambda tuple: tuple[2]);
    
    events_sorted = [event for event, _, _ in events_zip];
    duration_scalars = [duration for _, duration, _ in events_zip];
    
    total_slot_priority: float = 0;
    num_late: int = 0;
    event_sooness_penalty: float = 0;
    
    slot_index: int = 0;
    out_of_space: bool = False;
    
    # walk through each event, with each x
    for x_index, event in enumerate(events_sorted):
        if debug: print(f'current event: {event.id}');
                    
        total_minutes = event.calc_duration(duration_scalars[x_index]).total_seconds() / 60;
        duration: timedelta = timedelta(minutes=round(total_minutes / 15) * 15);
        
        if debug: print(f'duration: {duration}');
        
        # loop to find a slot
        while(True):
            
            # first check to see if out of bounds
            if slot_index >= len(slots) or out_of_space: 
                if debug: print(f'checking slot: OUT_OF_SPACE');
                out_of_space = True;
                num_late += 1;
                total_slot_priority += 5.01;
                break;
                
            if debug: print(f'checking slot: {slots[slot_index].id}');
            # next check to see if current slot has room for event
            if slots[slot_index].capacity < duration:
                # move to next slot if previous too small
                slot_index += 1;
            
            # there is room, move on to schedule   
            else: break;
            
        # assign event to slot
        if not out_of_space:
            start: datetime = slots[slot_index].adj_start;
            
            end: datetime = start + duration;
            
            if debug: print(f'scheduling event: {event.id}');
            event.schedule_event(start=start, end=end);
            slots[slot_index].time_used += int(duration.total_seconds() // 60);

            # Calculate objectives
            total_slot_priority += slots[slot_index].priority;
            
            if slots[slot_index].adj_start + event.min_time > event.due_date:
                num_late += 10;
                
            days_till_due = (event.due_date - event.start).days;
            if debug: print(f'days: {days_till_due}');
                
            event_sooness_penalty += 1 - pow( np.e, 0.2 * (days_till_due));
            # if days_till_due <= 0: event_sooness_penalty += 10;
                    
    return (total_slot_priority, num_late, event_sooness_penalty, events, slots);

In [90]:
storage = SchedulerStorage();

class SchedulerProblem(ElementwiseProblem):
    def __init__(self, slots: List[Slot], events: List[Event], debug: bool = False):
        self.slots = slots;
        self.events = events;
        self.debug = debug;
        
        # Two variables per event -> the 0 to 1 scale of event.min to event.max time
        #                         -> the order to schedule events, 0 to 1
        n_var = len(events) * 2;
        
        n_obj = 2;
        n_constr = 1;
        
        # Decision variable bounds (0 to 1)
        xl = np.zeros(n_var);
        xu = np.ones(n_var);

        super().__init__(n_var=n_var, n_obj=n_obj, n_constr=n_constr, xl=xl, xu=xu);

    # ================== ASSUMPTIONS ==================== #
    # Slots are given in the order that they should be used
    
    def _evaluate(self, x, out, *args, **kwargs):
        
        total_slot_priority, num_late, sooness, _, _ = optimize_schedule(x, self.slots, self.events, self.debug);

        out["F"] = [total_slot_priority, sooness];
        out["G"] = [num_late];
        
        # reset objects
        for slot in self.slots:
            slot.time_used = 0;
            
        for event in self.events:
            event.is_scheduled = False;
            event.start = None;
            event.end = None;
        


In [91]:
scheduler = Scheduler(storage);
slots = scheduler.slots;
events = scheduler.events;

In [92]:
print(slots);
print(events);

[Slot(start=datetime.datetime(2025, 4, 15, 9, 15), end=datetime.datetime(2025, 4, 15, 16, 0), priority=1, id=27, time_used=0), Slot(start=datetime.datetime(2025, 4, 16, 11, 0), end=datetime.datetime(2025, 4, 16, 12, 0), priority=1, id=30, time_used=0), Slot(start=datetime.datetime(2025, 4, 16, 13, 0), end=datetime.datetime(2025, 4, 16, 14, 0), priority=1, id=32, time_used=0), Slot(start=datetime.datetime(2025, 4, 17, 9, 15), end=datetime.datetime(2025, 4, 17, 15, 0), priority=1, id=35, time_used=0), Slot(start=datetime.datetime(2025, 4, 18, 11, 0), end=datetime.datetime(2025, 4, 18, 12, 0), priority=1, id=38, time_used=0), Slot(start=datetime.datetime(2025, 4, 18, 13, 0), end=datetime.datetime(2025, 4, 18, 14, 0), priority=1, id=40, time_used=0), Slot(start=datetime.datetime(2025, 4, 15, 18, 0), end=datetime.datetime(2025, 4, 15, 20, 0), priority=2, id=28, time_used=0), Slot(start=datetime.datetime(2025, 4, 16, 16, 0), end=datetime.datetime(2025, 4, 16, 20, 0), priority=2, id=33, time_

In [93]:
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

problem = SchedulerProblem(slots, events);

algorithm = NSGA2(pop_size=100,
            sampling=FloatRandomSampling(),
            crossover=SBX(prob=1.0, eta=3.0),
            mutation=PM(prob=1.0, eta=3.0),
            eliminate_duplicates=False,
            );

termination = get_termination("n_gen", 100);

result = minimize(
    problem,
    algorithm,
    termination,
    verbose=False
);

print("Optimization Completed!");
print("Best Solutions:");
for i, solution in enumerate(result.X):
    print(solution, end=' - ');
    print(result.F[i]);
    
    

Optimization Completed!
Best Solutions:
[4.51472893e-02 3.18625498e-01 1.95969416e-01 1.97116050e-02
 5.76373128e-04 9.49360544e-01 3.01194509e-04 2.14903768e-02
 9.65755709e-01 7.68582741e-01 5.31669667e-01 6.32684403e-01
 7.91657676e-01 3.83561427e-01] - [  8.         -18.69939391]
[0.05404208 0.26444542 0.15913448 0.00964862 0.00603545 0.40576235
 0.01898471 0.02973901 0.89439063 0.71241368 0.25299102 0.56578905
 0.81954239 0.38046746] - [  8.         -18.69939391]
[0.05404208 0.2932386  0.15913448 0.0082854  0.0065999  0.20602595
 0.01848365 0.01931596 0.89431095 0.71241368 0.5312016  0.56578905
 0.81954239 0.47500561] - [  8.         -18.69939391]
[0.03371325 0.34693052 0.12541159 0.01920678 0.00774753 0.45073878
 0.00831356 0.14872429 0.96606149 0.72770874 0.17873059 0.58302191
 0.77643094 0.33924443] - [  8.         -18.69939391]
[0.05511833 0.29437494 0.14481384 0.0082854  0.00992513 0.31331512
 0.01864433 0.02208503 0.87281258 0.72325657 0.54048604 0.56578905
 0.81954239 0.393

In [94]:
chosen_solution = result.X[0];

_, _, _, opt_events, opt_slots = optimize_schedule(chosen_solution, slots, events, False);

In [95]:
scheduler.slots = opt_slots;
scheduler.events = opt_events;
scheduler.save_scheduled_events();